In [1]:
import pandas as pd
import pickle

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)
pd.set_option('display.precision', 3)

import psycopg2 as pg
import pandas.io.sql as pd_sql

In [2]:
connection_args = {
    'host': 'localhost',  
    'dbname': 'cps',    
    'port': 5432
}

conn = pg.connect(**connection_args)

# The following data is not considered in this model
                   Address', 'Administrator_Title', 'CPS_School_Profile', 
                   'City', 'Classification_Description', 'Closed_For_Enrollment_Date',
                  'College_Enrollment_Rate_Mean', 'College_Enrollment_Rate_School',
                  'Demographic_Description', 'Earliest_Drop_Off_Time', 'Facebook', 
                  'Fax', 'Fifth_Contact_Name', 'Fifth_Contact_Title', 'Finance_ID', 
                  'Fourth_Contact_Name', 'Fourth_Contact_Title', 'Freshman_Start_End_Time',
                  'Grades_Offered', 'Graduation_Rate_Mean', 'Is_Elementary_School', 
                  'Is_GoCPS_Elementary', 'Is_GoCPS_High_School', 'Is_GoCPS_PreK', 
                  'Is_High_School', 'Is_Middle_School', 'Is_Pre_School', 'Kindergarten_School_Day',
                  'Legacy_Unit_ID', 'Long_Name', 'Mean_ACT', 'Open_For_Enrollment_Date', 
                  'Overall_Rating', 'Phone', 'Pinterest', 'PreK_School_Day', 'PreSchool_Inclusive', 
                  'Preschool_Instructional', 'Primary_Category', 'Rating_Statement', 'Rating_Status', 
                  'Refugee_Services', 'School_Type','Secondary_Contact', 'Secondary_Contact_Title', 
                  'Seventh_Contact_Name', 'Seventh_Contact_Title', 'Short_Name', 'Short_Name', 
                  'Significantly_Modified', 'Sixth_Contact_Name', 'Sixth_Contact_Title', 'State',
                  'Statistics_Description', 'Summary', 'Third_Contact_Name', 'Third_Contact_Title', 
                  'Title_1_Eligible', 'Twitter', 'Visual_Impairments', 'Website',
                  'Youtube', 'Zip_x', 'Attendance_Boundaries','Average_ACT_School', 
                   'Bilingual_Services','Hard_Of_Hearing', 'Is_GoCPS_Participant',
                  'School_Latitude', 'School_Longitude', 'Average_ACT_School', 'Location',
                   'Transportation_Bus','Transportation_Metra', 'After_School_Hours','School_Hours'

In [3]:
target_columns_1617 = '''School_Id,
                      Administrator,
                      Grades_Offered_All,
                      Student_Count_Total,
                      Student_Count_Low_Income,
                      Student_Count_Special_Ed,
                      Student_Count_English_Learners,
                      Student_Count_Black,
                      Student_Count_Hispanic,
                      Student_Count_White,
                      Student_Count_Asian,
                      Student_Count_Native_American,
                      Student_Count_Other_Ethnicity,
                      Student_Count_Asian_Pacific_Islander,
                      Student_Count_Multi,
                      Student_Count_Hawaiian_Pacific_Islander,
                      Student_Count_Ethnicity_Not_Available,
                      ADA_Accessible,
                      Dress_Code,
                      Classroom_Languages,
                      Transportation_El,
                      Graduation_Rate_School,
                      School_Year,
                      Zip
                '''

In [4]:
#no ADA accesible field for 1718, zip not differentiated for 1718
target_columns_1718 = '''School_Id,
                      Administrator,
                      Grades_Offered_All,
                      Student_Count_Total,
                      Student_Count_Low_Income,
                      Student_Count_Special_Ed,
                      Student_Count_English_Learners,
                      Student_Count_Black,
                      Student_Count_Hispanic,
                      Student_Count_White,
                      Student_Count_Asian,
                      Student_Count_Native_American,
                      Student_Count_Other_Ethnicity,
                      Student_Count_Asian_Pacific_Islander,
                      Student_Count_Multi,
                      Student_Count_Hawaiian_Pacific_Islander,
                      Student_Count_Ethnicity_Not_Available,
                      Dress_Code,
                      Classroom_Languages,
                      Transportation_El,
                      Graduation_Rate_School,
                      School_Year
                '''

In [5]:
cur = conn.cursor()
query_1617 = ('''SELECT %s FROM allschools_1617 WHERE is_high_school = 'Y' '''%target_columns_1617);
df_hs_201617 = pd_sql.read_sql(query_1617, conn)
query_1718 = ('''SELECT %s FROM allschools_1718 WHERE is_high_school = 'Y' ''' %target_columns_1718);
df_hs_201718 = pd_sql.read_sql(query_1718, conn)
dfs=[df_hs_201617,df_hs_201718]
df = pd.concat(dfs)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys


## Fill in ADA accessible and create ADA dummie

In [6]:
# 2016-17 has ADA accessibility data, and I will assume it is the same for 17-18.

In [7]:
ada = df[df['school_year'] == 'School Year 2016-2017'][['ada_accessible', 'school_id']]
df = pd.merge(df, ada, on = 'school_id' )
df.drop(columns = 'ada_accessible_x', inplace = True)
df = pd.merge(df, ada, on = 'school_id' )
df.drop(columns = 'ada_accessible', inplace = True)
df = pd.concat((df, pd.get_dummies(df['ada_accessible_y'])), axis=1)
#having [0,0] for fully and generally accessible implies no or unknown accessibility
df.drop(columns = ['No/unknown accessibility','ada_accessible_y'], inplace=True)

## Demographic Percentage Transformation

In [8]:
list(df)

['administrator',
 'classroom_languages',
 'dress_code',
 'grades_offered_all',
 'graduation_rate_school',
 'school_id',
 'school_year',
 'student_count_asian',
 'student_count_asian_pacific_islander',
 'student_count_black',
 'student_count_english_learners',
 'student_count_ethnicity_not_available',
 'student_count_hawaiian_pacific_islander',
 'student_count_hispanic',
 'student_count_low_income',
 'student_count_multi',
 'student_count_native_american',
 'student_count_other_ethnicity',
 'student_count_special_ed',
 'student_count_total',
 'student_count_white',
 'transportation_el',
 'zip',
 'Fully Accessible',
 'Generally accessible']

In [9]:
#Convert student demographic counts to percent of total population
print(list(df))
demog_headings = ['student_count_asian', 'student_count_asian_pacific_islander', 'student_count_black', 
                  'student_count_english_learners', 'student_count_ethnicity_not_available', 
                  'student_count_hawaiian_pacific_islander', 'student_count_hispanic', 
                  'student_count_low_income', 'student_count_multi', 'student_count_native_american', 
                  'student_count_other_ethnicity', 'student_count_special_ed', 'student_count_white']
for dem in demog_headings:
    dem_per_heading = dem + '_perc'
    df[dem_per_heading] = df[dem]/df['student_count_total']
    df.drop(columns = dem, inplace=True)

['administrator', 'classroom_languages', 'dress_code', 'grades_offered_all', 'graduation_rate_school', 'school_id', 'school_year', 'student_count_asian', 'student_count_asian_pacific_islander', 'student_count_black', 'student_count_english_learners', 'student_count_ethnicity_not_available', 'student_count_hawaiian_pacific_islander', 'student_count_hispanic', 'student_count_low_income', 'student_count_multi', 'student_count_native_american', 'student_count_other_ethnicity', 'student_count_special_ed', 'student_count_total', 'student_count_white', 'transportation_el', 'zip', 'Fully Accessible', 'Generally accessible']


## Language Count Transform

In [10]:
df.classroom_languages.value_counts()

Spanish                                                                   121
French, Spanish                                                            45
Spanish, Spanish for Heritage Speakers                                     31
French, Spanish, Spanish for Heritage Speakers                             13
Chinese, French, Spanish                                                    8
Mandarin, Spanish                                                           7
Chinese, Spanish                                                            4
American Sign Language                                                      4
Chinese, French, Japanese, Latin, Spanish                                   4
Chinese, French, German, Japanese, Latin, Spanish                           2
Arabic, Chinese                                                             2
American Sign Language, French, Spanish, Spanish for Heritage Speakers      2
                                                                

In [11]:
df.classroom_languages.isna().sum()

87

In [12]:
df_languages = df[['school_id', 'classroom_languages', 'school_year']]
#the data appears to show the same value for languages across the two years
#so the data will be transformed on one year to prevent duplication at time of merge
df_languages = df_languages[df_languages['school_year'] == 'School Year 2016-2017']

In [13]:
#Because this is a comma count, the values with zero commas have 1 language, 
#one comma have 2, etc.
df_languages.classroom_languages.str.count(',').value_counts()

0.0    64
1.0    50
2.0    14
3.0     5
4.0     3
5.0     2
9.0     1
7.0     1
Name: classroom_languages, dtype: int64

In [14]:
#NaN will be treated as 0 in the language count. 
#Since language count will be performed with a comma count,
#set nan to 15 and reset to 0 after the count is performed.
df_languages.fillna(',,,,,,,,,,,,,,', inplace=True)
df_languages['Classroom_Languages_count'] = (
                                            df_languages['classroom_languages'].str.count(',') 
                                            + 1
                                            )
df_languages['Classroom_Languages_count'].loc[df_languages['Classroom_Languages_count'] == 15] = 0

/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [15]:
#count values to show that now there are 44 schools with 0 languages
df_languages['Classroom_Languages_count'].value_counts()

1     64
2     50
0     44
3     14
4      5
5      3
6      2
10     1
8      1
Name: Classroom_Languages_count, dtype: int64

In [17]:
df_languages.sort_values('Classroom_Languages_count', ascending = False)
df_languages.drop(columns = ['classroom_languages', 'school_year'], inplace=True)
df = pd.merge(df, df_languages, on='school_id')

In [20]:
df.head()

,administrator,classroom_languages,dress_code,grades_offered_all,graduation_rate_school,school_id,school_year,student_count_total,transportation_el,zip,Fully Accessible,Generally accessible,student_count_asian_perc,student_count_asian_pacific_islander_perc,student_count_black_perc,student_count_english_learners_perc,student_count_ethnicity_not_available_perc,student_count_hawaiian_pacific_islander_perc,student_count_hispanic_perc,student_count_low_income_perc,student_count_multi_perc,student_count_native_american_perc,student_count_other_ethnicity_perc,student_count_special_ed_perc,student_count_white_perc,Classroom_Languages_count
0,Ms. Irma Plaxico,None,N,"9,10,11,12",NaN,400124,School Year 2016-2017,124,Green,60616.0,0,0,0.000,0.0,0.944,0.016,0.000,0.0,0.056,0.976,0.000,0.000,0.0,0.234,0.000,0
1,Ms. Irma Plaxico,None,N,"9,10,11,12",NaN,400124,School Year 2017-2018,153,Green,NaN,0,0,0.000,0.0,0.974,0.007,0.000,0.0,0.020,0.634,0.000,0.000,0.0,0.242,0.007,0
2,Juan Carlos Ocon,"French, Spanish",N,"9,10,11,12",80.9,609764,School Year 2016-2017,1739,"Blue, Pink, Red",60608.0,0,1,0.003,0.0,0.033,0.190,0.006,0.0,0.945,0.954,0.002,0.003,0.0,0.162,0.007,2
3,Juan Carlos Ocon,"French, Spanish",N,"9,10,11,12",78.6,609764,School Year 2017-2018,1706,"Blue, Pink, Red",NaN,0,1,0.006,0.0,0.031,0.201,0.008,0.0,0.943,0.626,0.004,0.004,0.0,0.162,0.006,2
4,Ms. Carrie Spitz,"Russian, Spanish",Y,"9,10,11,12",90.3,400054,School Year 2016-2017,992,"Blue, Brown",60639.0,0,1,0.001,0.0,0.030,0.121,0.000,0.0,0.953,0.944,0.002,0.005,0.0,0.107,0.009,2


## Merge Income Data Per Zip for 16/17 and 17/18

In [ ]:
df_2016 = df[df['School_Year'] == 'School Year 2016-2017']
df_2016_income = pd.read_csv('data/meanHHincome_2016.csv', header=None)
df_2016_income.columns = ['Zip_y', 'Zip_Mean_Income']
df_2016 = pd.merge(df_2016, df_2016_income, on = 'Zip_y' )

In [ ]:
df_2017 = df[df['School_Year'] == 'School Year 2017-2018']
df_2017_income = pd.read_csv('data/meanHHincome_2017.csv', header=None)
df_2017_income.columns = ['Zip_y', 'Zip_Mean_Income']
df_2017 = pd.merge(df_2017, df_2017_income, on = 'Zip_y' )

In [ ]:
df = df_2016.append(df_2017)

In [ ]:
print(list(df))

In [ ]:
def replace_comma(row):
    return float(row['Zip_Mean_Income'].replace(',', ''))

df['Zip_Mean_Income'] = df.apply(lambda row: replace_comma(row), axis =1) 


## El Dummie Transform

In [ ]:
el_df = df[['School_ID', 'Transportation_El', 'School_Year']]

In [ ]:
el_df.head(10)

In [ ]:
#Again, like languages, looks like El values are consistent across years.
el_df = el_df[el_df['School_Year'] == 'School Year 2017-2018']

In [ ]:
el_df['Transportation_El'].value_counts()

In [ ]:
#replace NaN with no_el so that with dummy variable I can drop No_El
el_df.fillna(value = 'No_El', inplace=True)

In [ ]:
el_df

In [ ]:
el_dummies = el_df['Transportation_El'].str.get_dummies(sep = ', ')

In [ ]:
el_dummies

In [ ]:
el_dummies.drop(columns = 'No_El', inplace=True)

In [ ]:
el_df = pd.merge(el_df,el_dummies, left_index=True, right_index=True)

In [ ]:
el_df.head()

In [ ]:
el_df.drop(columns = ['Transportation_El', 'School_Year'], inplace=True)

In [ ]:
#decide which column to drop for dummies based on column count
line_list = ['Blue','Brown','Green','Orange','Pink','Purple','Red']
for line in line_list:
    print(el_df[line].value_counts())

In [ ]:
df = pd.merge(df, el_df, on='School_ID')

In [ ]:
df.drop(columns = ['Transportation_El'], inplace=True)

In [ ]:
df.info()

## Make Dress Code Binary

In [ ]:
df["Dress_Code_dummie"] = pd.get_dummies(df['Dress_Code'], drop_first = True)


In [ ]:
df.drop(columns='Dress_Code', inplace=True)

In [ ]:
df.head()

## Grades offered count

In [ ]:
df.Grades_Offered_All.value_counts()

In [ ]:
df_grades = df[['Grades_Offered_All', 'School_ID']]

                                            

In [ ]:
df_grades['grades_offered_count'] = (
                                            df_grades['Grades_Offered_All'].str.count(',') 
                                            + 1
                                            )
  
    
    

In [ ]:
df_grades.grades_offered_count.value_counts()

In [ ]:
df_grades.drop(columns = ['Grades_Offered_All'], inplace=True)

In [ ]:
df_grades.drop_duplicates('School_ID', inplace=True)
df_grades.sort_values('grades_offered_count')

In [ ]:
df = pd.merge(df, df_grades, on='School_ID')

In [ ]:
df.drop(columns = ['Classroom_Languages', 'Grades_Offered_All'], inplace=True)

In [ ]:
df.head()

## Charter Dummie

In [ ]:
df_networks = df[['School_ID', 'Network', 'School_Year']]

In [ ]:
df_networks.Network.value_counts()

In [ ]:
df_networks2017 = df_networks[df_networks['School_Year']=='School Year 2017-2018']

In [ ]:
df_networks2017.info()

In [ ]:
def charter(row):
    if row['Network'] == 'Charter':
        return 1
    else:
        return 0


In [ ]:
df_networks2017['charter'] = df_networks2017.apply(lambda row: charter(row), axis=1)
df_networks2017.drop(columns=['School_Year', 'Network'], inplace=True)

In [ ]:
df_networks2017.head()

In [ ]:
df_networks2017.charter.value_counts()

In [ ]:
df = pd.merge(df, df_networks2017, on='School_ID')

In [ ]:
df.drop(columns='Network', inplace=True)

## Admin. Gender

In [ ]:
import re

In [ ]:
df_admin = df[['Administrator', 'School_ID']]

In [ ]:
df_admin.drop_duplicates('School_ID')

In [ ]:
gender = re.compile(r'Mrs|Ms|Mr')
#female_title = re.compile(r'Mrs|Ms')

In [ ]:
df_admin['gender_marker'] = df_admin['Administrator'].astype(str).str.match(gender)
df_admin.drop_duplicates('School_ID', inplace=True)


In [ ]:
df_admin.gender_marker.value_counts()

In [ ]:
male_marker = re.compile(r'Mr|Juan|Richard|Ali|Kevin|Douglas|Raul|Victor|Abdul|Charles|Antonio|Brian|Francisco|\
        Sheldon|Michael|Stephen|Peter|Gregory|Trent|Myron|Gerald|Elias|Octavio|Matthew|\
        David|Leonard|Ferdinand|Fernando|Mark|Patrick|George|Wayne|Anthony|William|\
        Stephen|Timothy|Paul')
female_marker = re.compile(r'Mrs|Ms|Dr. Hillyn|Sharnette|Tressie|Leticia|Priscilla|Joyce|Stephanie|Tanya|Veronica|Kathy|\
         Sandra|Torry|Stephanie|Carolyn|Milena|Vanesa|Breanda|Laura|Kelly|Anna|Nancy|\
         Tamika|Janice|Mary|Shanele|Falilat|Dr.Femi|Noel|Tawanna|Tonya|Sandra|Dr. Vanesa|Tamika')
print(df_admin[df_admin['gender_marker']==False].Administrator.values)


In [ ]:
df_admin['gender_f'] = df_admin['Administrator'].astype(str).str.match(female_marker)

In [ ]:
df_admin[df_admin['gender_f'] == False].values

In [ ]:
df_admin.gender_f.value_counts()

In [ ]:
list(df_admin)

In [ ]:
df_admin.drop(columns = ['Administrator', 'gender_marker'], inplace=True)

In [ ]:
df = pd.merge(df, df_admin, on='School_ID')


In [ ]:
def gender(row):
    if row['gender_f'] == True:
        return 1
    else:
        return 0

In [ ]:
df['gender_f_bn'] = df.apply(lambda row: gender(row), axis=1)

In [ ]:
df.head(5)

In [ ]:
df.drop(columns = ['Administrator', 'gender_f'], inplace = True)


In [ ]:
df.head()

## Reflect and log transform graduation rate

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
plot = sns.distplot(df['Graduation_Rate_School'])
plt.title('Graduation Rates in CPS High Schools - 2017/2018')
plt.xlabel('Graduation Rate')

In [ ]:
df['Grad_Rate_Reflected'] = 100 - df['Graduation_Rate_School']

In [ ]:
df['log_grad_rate'] = np.log(df['Grad_Rate_Reflected'])

In [ ]:
#Reflecting and log transforming the data makes it more normal.
sns.distplot(df['log_grad_rate'])

In [ ]:
df.drop(columns = ['Graduation_Rate_School', 'Grad_Rate_Reflected'], inplace=True)

In [ ]:
list(df)

In [ ]:
len(list(df))

## Pickle 29 Feature Columns

In [ ]:
df.drop(columns = ['School_ID', 'Zip_y', 'School_Year'], inplace=True)

In [ ]:
with open('data/pickles/29feat_formodeling.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

## Can't Get Working: After School Hours

In [ ]:
#df_as_hours = df[['After_School_Hours', 'School_ID']]

In [ ]:
#print(df_as_hours['After_School_Hours'].value_counts())

In [ ]:
#df_as_hours['as_start'] = df_as_hours['After_School_Hours'].astype(str).str[0:4]

In [ ]:
#df_as_hours.head(30)

In [ ]:
#df_as_hours.as_start.value_counts()

In [ ]:
#df_as_hours['no_pm'] = df_as_hours.After_School_Hours.str.replace('P.M.', '')

In [ ]:
#df_as_hours['no_pm'] = df_as_hours.no_pm.str.replace('PM', '')

In [ ]:
#df_as_hours['as_end'] = df_as_hours['no_pm'].astype(str).str[-5:]

In [ ]:
#df_as_hours.as_end = df_as_hours.as_end.str.replace('-', '')

In [ ]:
#df_as_hours.as_end.value_counts()

In [ ]:
#import re

In [ ]:
#after_end_dash = re.compile(r"\-(.*)")
#after_end_to = re.compile(r"t(.*)")

In [ ]:
'''
for hour_range in df['After_School_Hours'].items():
    #print(hour_range[1])
    if '-' in str(hour_range[1]):
        df_as_hours['as_stop'] = after_end_dash.search(hour_range[1])
    elif 'to' in str(hour_range[1]):
            df_as_hours['as_stop']=after_end_to.search(hour_range[1])

'''



In [ ]:
#df_as_hours.as_stop.value_counts()